<a href="https://colab.research.google.com/github/PrasannaSgkumar/ChatBot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
import json
with open('/content/intents.JSON') as json_data:
  intents=json.load(json_data)

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# intents = json.loads(open('C:/Simplilearn/Python/Python projects/chatbot using python/chatbot/intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append(bag + output_row)

random.shuffle(training)
training = np.array(training)

train_x = training[:, :len(words)]
train_y = training[:, len(words):]

In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/200
3/3 [==============================] - 1s 7ms/step - loss: 1.4429 - accuracy: 0.2667
Epoch 2/200
3/3 [==============================] - 0s 4ms/step - loss: 1.4388 - accuracy: 0.2000
Epoch 3/200
3/3 [==============================] - 0s 4ms/step - loss: 1.3330 - accuracy: 0.4000
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 1.2550 - accuracy: 0.5333
Epoch 5/200
3/3 [==============================] - 0s 4ms/step - loss: 1.3025 - accuracy: 0.4000
Epoch 6/200
3/3 [==============================] - 0s 4ms/step - loss: 0.9824 - accuracy: 0.6667
Epoch 7/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0427 - accuracy: 0.6000
Epoch 8/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0756 - accuracy: 0.5333
Epoch 9/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0377 - accuracy: 0.6667
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 0.8927 - accuracy: 0.6667
Epoch 11/200
3/3 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

lemmatizer = WordNetLemmatizer()
import json
with open('/content/intents.JSON') as json_data:
  intents=json.load(json_data)
#intents = json.loads(open('C:/Simplilearn/Python/Python projects/chatbot using python/chatbot/intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [8]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = "I'm sorry, but I didn't understand that."
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("GO! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


GO! Bot is running!
hi
1/1 [==============================] - 0s 119ms/step
Good to see you again
who are you
1/1 [==============================] - 0s 27ms/step
Hello


KeyboardInterrupt: Interrupted by user